In [1]:
import pandas as pd
import numpy as np
import glob
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from dateutil import parser
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
import joblib, os

In [5]:
class Processing:
    def __init__(self,data_path,label_path):
        self.data = data_path
        self.label= label_path
    def process(data_path,label_path):
        scores_df = pd.read_csv(label_path)

        #hit scores
        hit_df = scores_df[["subject","date","crossing number","run","limb","avg_human_hit"]]

        #miss
        miss_df = scores_df[["subject","date","crossing number","run","limb","avg_human_miss"]]

        #steps
        step_df = scores_df[["subject","date","crossing number","run","limb","avg_human_steps"]]

        #score extraction
        dom_f_hit = hit_df.loc[hit_df["limb"]=="Dominant Front"]
        dom_f_miss = miss_df.loc[miss_df["limb"]=="Dominant Front"]
        dom_f_step = step_df.loc[step_df["limb"]=="Dominant Front"]

        ndom_f_hit = hit_df.loc[hit_df["limb"]=="Nondominant Front"]
        ndom_f_miss = miss_df.loc[miss_df["limb"]=="Nondominant Front"]
        ndom_f_step = step_df.loc[step_df["limb"]=="Nondominant Front"]

        dom_b_hit = hit_df.loc[hit_df["limb"]=="Dominant Back"]
        dom_b_miss = miss_df.loc[miss_df["limb"]=="Dominant Back"]
        dom_b_step = step_df.loc[step_df["limb"]=="Dominant Back"]

        ndom_b_hit = hit_df.loc[hit_df["limb"]=="Nondominant Back"]
        ndom_b_miss = miss_df.loc[miss_df["limb"]=="Nondominant Back"]
        ndom_b_step = step_df.loc[step_df["limb"]=="Nondominant Back"]
        dom_f_hit_x = []
        dom_f_miss_x = []
        dom_f_step_x =[]

        ndom_f_hit_x = []
        ndom_f_miss_x = []
        ndom_f_step_x = []

        dom_b_hit_x = []
        dom_b_miss_x = []
        dom_b_step_x = []

        ndom_b_hit_x = []
        ndom_b_miss_x = []
        ndom_b_step_x = []

        #sets of labels
        dom_f_hit_y = []
        dom_f_miss_y = []
        dom_f_step_y =[]

        ndom_f_hit_y = []
        ndom_f_miss_y = []
        ndom_f_step_y = []

        dom_b_hit_y = []
        dom_b_miss_y = []
        dom_b_step_y = []

        ndom_b_hit_y = []
        ndom_b_miss_y = []
        ndom_b_step_y = []

        lengths = []
        rat_folder = glob.glob(data_path)
        
        for file in rat_folder:
            #rat tracking file
            #name of the rung file
            rung_name_list = file.split("/")[-1].split("_")[0:8]+["LadderWalkMar12shuffle1","450000.h5"]
            rung_file = '_'.join(rung_name_list)
            #open the file
            rat_df = pd.read_hdf(file)["DLC_resnet50_LadderWalkFeb13shuffle1_450000"]
            #properties of the file
            subject = file.split("/")[0]
            date_raw = rung_name_list[1]
            date = parser.parse(date_raw).date().strftime("%Y-%m-%d")
            run = rung_name_list[2]
            crossing = [int(s) for s in rung_name_list[3] if s.isdigit()][0]
            #
            dom_f_hit_score = dom_f_hit[(dom_f_hit["subject"]==subject) & (dom_f_hit["date"]==date) & (dom_f_hit["run"]==run)].reset_index()
            dom_f_miss_score = dom_f_miss[(dom_f_miss["subject"]==subject) & (dom_f_miss["date"]==date) & (dom_f_miss["run"]==run)].reset_index()
            dom_f_step_score = dom_f_step[(dom_f_step["subject"]==subject) & (dom_f_step["date"]==date) & (dom_f_step["run"]==run)].reset_index()

            ndom_f_hit_score = ndom_f_hit[(ndom_f_hit["subject"]==subject) & (ndom_f_hit["date"]==date) & (ndom_f_hit["run"]==run)].reset_index()
            ndom_f_miss_score = ndom_f_miss[(ndom_f_miss["subject"]==subject) & (ndom_f_miss["date"]==date) & (ndom_f_miss["run"]==run)].reset_index()
            ndom_f_step_score = ndom_f_step[(ndom_f_step["subject"]==subject) & (ndom_f_step["date"]==date) & (ndom_f_step["run"]==run)].reset_index()

            dom_b_hit_score = dom_b_hit[(dom_b_hit["subject"]==subject) & (dom_b_hit["date"]==date) & (dom_b_hit["run"]==run)].reset_index()
            dom_b_miss_score = dom_b_miss[(dom_b_miss["subject"]==subject) & (dom_b_miss["date"]==date) & (dom_b_miss["run"]==run)].reset_index()
            dom_b_step_score = dom_b_step[(dom_b_step["subject"]==subject) & (dom_b_step["date"]==date) & (dom_b_step["run"]==run)].reset_index()

            ndom_b_hit_score = ndom_b_hit[(ndom_b_hit["subject"]==subject) & (ndom_b_hit["date"]==date) & (ndom_b_hit["run"]==run)].reset_index()
            ndom_b_miss_score = ndom_b_miss[(ndom_b_miss["subject"]==subject) & (ndom_b_miss["date"]==date) & (ndom_b_miss["run"]==run)].reset_index()
            ndom_b_step_score = ndom_b_step[(ndom_b_step["subject"]==subject) & (ndom_b_step["date"]==date) & (ndom_b_step["run"]==run)].reset_index()

            #join the rat and rung dataframes
            df = rat_df
            df_cols = df.columns.tolist()
            df_temp = df

            #remove where median likelihood is low
            #df_low_like = df.columns[-df_temp[df_temp.columns.get_level_values(0).unique()].median().ge(0.2)].get_level_values(0).tolist()
            #df_cols_up = [x for x in df_cols if x not in df_low_like ]

            #df = rat_df[df_cols_up]
            if df.shape[1] == 0:
                continue
            #if len(dom_f_low_like) >0:
                #print(dom_f_low_like)
            df = df.drop('likelihood', axis=1, level=1)

            #scale data
            ####Uses the values function of pandas - converts any dataframe to an array
            data_for_scaling = df.values
            #### the scaling object
            scaler = MinMaxScaler()
            #We will use fit_transform here - we want to actually scale this data, not use the scaler on a different dataset
            scaled_data = scaler.fit_transform(data_for_scaling)

            if len(scaled_data) < 356:
                newlength = (356-len(scaled_data))
                zero = np.zeros((newlength,6))
                scaled_temp = pd.DataFrame(scaled_data).append(pd.DataFrame(zero),ignore_index=True)
                scaled_temp = scaled_temp.fillna(0)
                scaled_data2 = scaled_temp.values
            else:
                scaled_data2 = scaled_data

            video_data = scaled_data2.flatten()
            #lengths.append([len(video_data)])
            if len(dom_f_hit_score) !=0:
                dom_f_hit_y.append(dom_f_hit_score["avg_human_hit"][0])
                dom_f_hit_x.append(video_data)
            else:
                print("Dom Front Hit"+"Missing scores: "+subject + " " + date + " " + run)
            if len(dom_f_miss_score) != 0:
                dom_f_miss_y.append(dom_f_miss_score["avg_human_miss"][0])
                dom_f_miss_x.append(video_data)
            else:
                print("Dom Front Miss"+"Missing scores: "+subject + " " + date + " " + run)
            if len(dom_f_step_score) != 0:
                dom_f_step_y.append(dom_f_step_score["avg_human_steps"][0])
                dom_f_step_x.append(video_data)
            else:
                print("Dom Front Step"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_f_hit_score) != 0:
                ndom_f_hit_y.append(ndom_f_hit_score["avg_human_hit"][0])
                ndom_f_hit_x.append(video_data)
            else:
                print("Nondom Front Hit"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_f_miss_score) != 0:
                ndom_f_miss_y.append(ndom_f_miss_score["avg_human_miss"][0])
                ndom_f_miss_x.append(video_data)
            else:
                print("Nondom Front Miss"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_f_step_score) != 0:
                ndom_f_step_y.append(ndom_f_step_score["avg_human_steps"][0])
                ndom_f_step_x.append(video_data)
            else:
                print("Nondom Front Step"+"Missing scores: "+subject + " " + date + " " + run)
            if len(dom_b_hit_score) != 0:
                dom_b_hit_y.append(dom_b_hit_score["avg_human_hit"][0])
                dom_b_hit_x.append(video_data)
            else:
                print("Dom Back Hit"+"Missing scores: "+subject + " " + date + " " + run)
            if len(dom_b_miss_score) != 0:
                dom_b_miss_y.append(dom_b_miss_score["avg_human_miss"][0])
                dom_b_miss_x.append(video_data)
            else:
                print("Dom Back Miss"+"Missing scores: "+subject + " " + date + " " + run)
            if len(dom_b_step_score) != 0:
                dom_b_step_y.append(dom_b_step_score["avg_human_steps"][0])
                dom_b_step_x.append(video_data)
            else:
                print("Dom Back Step"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_b_hit_score) != 0:
                ndom_b_hit_y.append(ndom_b_hit_score["avg_human_hit"][0])
                ndom_b_hit_x.append(video_data)
            else:
                print("Nondom Back Hit"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_b_miss_score) != 0:
                ndom_b_miss_y.append(ndom_b_miss_score["avg_human_miss"][0])
                ndom_b_miss_x.append(video_data)
            else:
                print("Nondom Back Miss"+"Missing scores: "+subject + " " + date + " " + run)
            if len(ndom_b_step_score) != 0:
                ndom_b_step_y.append(ndom_b_step_score["avg_human_steps"][0])
                ndom_b_step_x.append(video_data)
            else:
                print("Nondom Back Step"+"Missing scores: "+subject + " " + date + " " + run)
                continue
        return [dom_f_hit_x, dom_f_miss_x, dom_f_step_x, ndom_f_hit_x, ndom_f_miss_x, ndom_f_step_x,
    dom_b_hit_x, dom_b_miss_x, dom_b_step_x, ndom_b_hit_x, ndom_b_miss_x, ndom_b_step_x],[dom_f_hit_y, dom_f_miss_y, dom_f_step_y, ndom_f_hit_y, ndom_f_miss_y, ndom_f_step_y, dom_b_hit_y,
    dom_b_miss_y, dom_b_step_y, ndom_b_hit_y, ndom_b_miss_y, ndom_b_step_y]


In [3]:
class Scoring:
    def __init__(self):
        self = self
    def forest(X,y,rf_name,data_path,project_name, njobs=None):
        data = np.array(X)
        labels = np.array(y)
        train_features, test_features, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, random_state = 42)
        # Instantiate model with n decision trees
        rf = RandomForestRegressor(n_estimators = 1000, random_state = 42,n_jobs=njobs)# Train the model on training data
        rf.fit(train_features, train_labels)
        # Use the forest's predict method on the test data
        predictions = rf.predict(test_features)# Calculate the absolute errors
        errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
        # Calculate mean absolute percentage error (MAPE)
        mape = 100 * (errors / test_labels)# Calculate and display accuracy
        accuracy = 100 - np.mean(mape)
        joblib.dump(rf, rf_name+".joblib", compress=0) 
        print(f"Uncompressed Random Forest: {np.round(os.path.getsize(rf_name+'.joblib') / 1024 / 1024, 2) } MB")
        
        rat_folder = glob.glob(data_path)
        df = pd.read_hdf(rat_folder[0])[project_name]
        df = df.drop('likelihood', axis=1, level=1)
        feats = {} # a dict to hold feature_name: feature_importance
        for feature, importance in zip(df.columns, rf.feature_importances_):
            feats[feature] = importance #add the name/value pair 
        importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
        return accuracy, importances.sort_values(by="Gini-importance",ascending=False)

In [6]:
datas = Processing.process("*/dlc_output_resnet50/*.h5","LW_Manual_scores_for_ICC_2020-05-20.csv")

Nondom Front HitMissing scores: MC70 2019-04-11 R5
Nondom Front MissMissing scores: MC70 2019-04-11 R5
Nondom Front StepMissing scores: MC70 2019-04-11 R5
Dom Back HitMissing scores: MC70 2019-04-11 R5
Dom Back MissMissing scores: MC70 2019-04-11 R5
Dom Back StepMissing scores: MC70 2019-04-11 R5
Nondom Back HitMissing scores: MC70 2019-04-11 R5
Nondom Back MissMissing scores: MC70 2019-04-11 R5
Nondom Back StepMissing scores: MC70 2019-04-11 R5
Dom Front HitMissing scores: MC70 2019-03-14 L4
Dom Front MissMissing scores: MC70 2019-03-14 L4
Dom Front StepMissing scores: MC70 2019-03-14 L4
Nondom Front HitMissing scores: MC70 2019-03-14 L4
Nondom Front MissMissing scores: MC70 2019-03-14 L4
Nondom Front StepMissing scores: MC70 2019-03-14 L4
Dom Back HitMissing scores: MC70 2019-03-14 L4
Dom Back MissMissing scores: MC70 2019-03-14 L4
Dom Back StepMissing scores: MC70 2019-03-14 L4
Nondom Back HitMissing scores: MC70 2019-03-14 L4
Nondom Back MissMissing scores: MC70 2019-03-14 L4
Nondo

In [7]:
data_x = datas[0]
labels_y = datas[1]

In [8]:
keys = ["dominant_front_hit", "dominant_front_miss", "dominant_front_step", 
 "nondominant_front_hit", "nondominant_front_miss", "nondominant_front_step",
    "dominant_back_hit", "dominant_back_miss", "dominant_back_step", 
 "nondominant_back_hit", "nondominant_back_miss", "nondominant_back_step"]

In [9]:
for i,k in enumerate(keys):
    print(i,k)
    acc, feat = Scoring.forest(data_x[i],labels_y[i],k,"*/dlc_output_resnet50/*.h5","DLC_resnet50_LadderWalkFeb13shuffle1_450000",njobs=6)
    print(acc,feat.head())

0 dominant_front_hit
Uncompressed Random Forest: 7.3 MB
81.30436218714236                    Gini-importance
(left fingers, y)         0.002549
(hip, y)                  0.002072
(left elbow, y)           0.002050
(right ankle, y)          0.001814
(left wrist, y)           0.001277
1 dominant_front_miss


<ipython-input-3-be73539821f5>:15: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_labels)# Calculate and display accuracy


Uncompressed Random Forest: 6.34 MB
-inf                    Gini-importance
(right eye, y)            0.002145
(left fingers, y)         0.001830
(left wrist, y)           0.001325
(nose, y)                 0.001162
(right wrist, y)          0.000720
2 dominant_front_step
Uncompressed Random Forest: 6.43 MB
86.88746867300493                    Gini-importance
(left fingers, y)         0.001107
(left ankle, y)           0.000691
(right ankle, y)          0.000575
(right elbow, y)          0.000561
(base of tail, y)         0.000506
3 nondominant_front_hit
Uncompressed Random Forest: 7.32 MB
85.36458768527504                  Gini-importance
(hip, y)                0.000527
(left wrist, y)         0.000477
(left eye, y)           0.000364
(right toes, y)         0.000348
(nose, y)               0.000339
4 nondominant_front_miss


<ipython-input-3-be73539821f5>:15: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_labels)# Calculate and display accuracy


Uncompressed Random Forest: 5.35 MB
-inf                   Gini-importance
(left eye, y)            0.006986
(left toes, y)           0.001851
(right ankle, y)         0.001585
(nose, y)                0.001339
(right eye, y)           0.001283
5 nondominant_front_step
Uncompressed Random Forest: 6.86 MB
85.5936721786659                   Gini-importance
(left wrist, y)          0.000394
(left elbow, y)          0.000281
(left eye, y)            0.000260
(left toes, y)           0.000252
(right ankle, y)         0.000239
6 dominant_back_hit
Uncompressed Random Forest: 6.2 MB
86.5488967689398                    Gini-importance
(hip, y)                  0.005808
(left elbow, y)           0.002031
(left fingers, y)         0.001823
(right wrist, y)          0.001373
(right ankle, y)          0.001327
7 dominant_back_miss


<ipython-input-3-be73539821f5>:15: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_labels)# Calculate and display accuracy


Uncompressed Random Forest: 2.58 MB
-inf                    Gini-importance
(left fingers, y)         0.001661
(right wrist, x)          0.000729
(left ankle, y)           0.000433
(right ankle, y)          0.000386
(left toes, y)            0.000373
8 dominant_back_step
Uncompressed Random Forest: 6.12 MB
89.65668524243952                    Gini-importance
(left elbow, y)           0.003228
(right ankle, y)          0.003166
(left toes, y)            0.000922
(right eye, y)            0.000845
(left fingers, y)         0.000711
9 nondominant_back_hit
Uncompressed Random Forest: 5.73 MB
87.8924065446048                    Gini-importance
(right ankle, y)          0.002038
(left fingers, y)         0.000978
(nose, y)                 0.000926
(hip, y)                  0.000880
(right toes, y)           0.000760
10 nondominant_back_miss


<ipython-input-3-be73539821f5>:15: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * (errors / test_labels)# Calculate and display accuracy


Uncompressed Random Forest: 2.16 MB
-inf                   Gini-importance
(left toes, y)           0.000480
(left ankle, y)          0.000438
(right elbow, y)         0.000382
(nose, y)                0.000377
(left wrist, y)          0.000323
11 nondominant_back_step
Uncompressed Random Forest: 5.83 MB
89.31786910697188                   Gini-importance
(left toes, y)           0.003919
(left elbow, y)          0.001645
(hip, y)                 0.000883
(left ankle, y)          0.000726
(right ankle, y)         0.000674
